In [24]:
import pandas as pd
import matplotlib
import numpy as np
import torch

https://colab.research.google.com/drive/13ErkLg5FZHIbnUGZRkKlL-9WNCNQPIow#scrollTo=H5t4_7HdjYEE


1.   pip install transformers 
2.   pip install wandb






In [9]:
df = pd.read_csv('train.csv')
df

,text_id,full_text,cohesion,syntax,vocabulary,phraseology,grammar,conventions
0,0016926B079C,I think that students would benefit from learn...,3.5,3.5,3.0,3.0,4.0,3.0
1,0022683E9EA5,When a problem is a change you have to let it ...,2.5,2.5,3.0,2.0,2.0,2.5
2,00299B378633,"Dear, Principal\n\nIf u change the school poli...",3.0,3.5,3.0,3.0,3.0,2.5
3,003885A45F42,The best time in life is when you become yours...,4.5,4.5,4.5,4.5,4.0,5.0
4,0049B1DF5CCC,Small act of kindness can impact in other peop...,2.5,3.0,3.0,3.0,2.5,2.5
...,...,...,...,...,...,...,...,...
3906,FFD29828A873,I believe using cellphones in class for educat...,2.5,3.0,3.0,3.5,2.5,2.5
3907,FFD9A83B0849,"Working alone, students do not have to argue w...",4.0,4.0,4.0,4.0,3.5,3.0
3908,FFDC4011AC9C,"""A problem is a chance for you to do your best...",2.5,3.0,3.0,3.0,3.5,3.0
3909,FFE16D704B16,Many people disagree with Albert Schweitzer's ...,4.0,4.5,4.5,4.0,4.5,4.5


In [10]:
df_cohesion = df[["text_id", "full_text", "cohesion"]]
df_cohesion

,text_id,full_text,cohesion
0,0016926B079C,I think that students would benefit from learn...,3.5
1,0022683E9EA5,When a problem is a change you have to let it ...,2.5
2,00299B378633,"Dear, Principal\n\nIf u change the school poli...",3.0
3,003885A45F42,The best time in life is when you become yours...,4.5
4,0049B1DF5CCC,Small act of kindness can impact in other peop...,2.5
...,...,...,...
3906,FFD29828A873,I believe using cellphones in class for educat...,2.5
3907,FFD9A83B0849,"Working alone, students do not have to argue w...",4.0
3908,FFDC4011AC9C,"""A problem is a chance for you to do your best...",2.5
3909,FFE16D704B16,Many people disagree with Albert Schweitzer's ...,4.0


In [11]:
df_cohesion['rating'] = np.zeros(len(df_cohesion))
for i in range(1, len(df_cohesion)):
    score = df_cohesion.loc[i, 'cohesion']
    if score >= 3.5:
        df_cohesion.loc[i, 'rating'] = 1
    if score <= 2.5:
        df_cohesion.loc[i, 'rating'] = -1

df_cohesion

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


,text_id,full_text,cohesion,rating
0,0016926B079C,I think that students would benefit from learn...,3.5,0.0
1,0022683E9EA5,When a problem is a change you have to let it ...,2.5,-1.0
2,00299B378633,"Dear, Principal\n\nIf u change the school poli...",3.0,0.0
3,003885A45F42,The best time in life is when you become yours...,4.5,1.0
4,0049B1DF5CCC,Small act of kindness can impact in other peop...,2.5,-1.0
...,...,...,...,...
3906,FFD29828A873,I believe using cellphones in class for educat...,2.5,-1.0
3907,FFD9A83B0849,"Working alone, students do not have to argue w...",4.0,1.0
3908,FFDC4011AC9C,"""A problem is a chance for you to do your best...",2.5,-1.0
3909,FFE16D704B16,Many people disagree with Albert Schweitzer's ...,4.0,1.0


In [12]:
df_cohesion_sentence = df_cohesion.assign(full_text_sentences=df_cohesion['full_text'].str.split(".").explode('full_text'))
df_cohesion_sentence = df_cohesion_sentence.explode('full_text_sentences')
df_cohesion_sentence = df_cohesion_sentence[df_cohesion_sentence["full_text_sentences"] != '']
df_cohesion_sentence = df_cohesion_sentence.drop(columns = ['full_text'])
df_cohesion_sentence

,text_id,cohesion,rating,full_text_sentences
0,0016926B079C,3.5,0.0,I think that students would benefit from learn...
1,0022683E9EA5,2.5,-1.0,taking only classes helps them because at the...
2,00299B378633,3.0,0.0,they will be comfortable at home
3,003885A45F42,4.5,1.0,\n\nThe hardest part of school is getting ready
4,0049B1DF5CCC,2.5,-1.0,you wake up go brush your teeth and go to you...
...,...,...,...,...
3906,FFD29828A873,2.5,-1.0,\n\nHe asked her how he could make up the poin...
3907,FFD9A83B0849,4.0,1.0,\n\nTwo weeks after that she assigned another ...
3908,FFDC4011AC9C,2.5,-1.0,\n\nShe took Generic_Name aside and said ' thi...
3909,FFE16D704B16,4.0,1.0,\n\nGeneric_Name listed so he asked somebody t...


In [13]:
sentences1 = df_cohesion_sentence["full_text_sentences"].values
labels = df_cohesion_sentence["rating"].values

In [15]:
import transformers
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

max_len = 0
for sent in sentences1:
    input_ids = tokenizer.encode(sent, add_special_tokens=True)
    max_len = max(max_len, len(input_ids))
  
print('Max paragraph length: ', max_len)


Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Max paragraph length:  396


In [44]:
sentences = df_cohesion["full_text"].values
labels = df_cohesion["rating"].values
sentences[0]

"I think that students would benefit from learning at home,because they wont have to change and get up early in the morning to shower and do there hair. taking only classes helps them because at there house they'll be pay more attention. they will be comfortable at home.\n\nThe hardest part of school is getting ready. you wake up go brush your teeth and go to your closet and look at your cloths. after you think you picked a outfit u go look in the mirror and youll either not like it or you look and see a stain. Then you'll have to change. with the online classes you can wear anything and stay home and you wont need to stress about what to wear.\n\nmost students usually take showers before school. they either take it before they sleep or when they wake up. some students do both to smell good. that causes them do miss the bus and effects on there lesson time cause they come late to school. when u have online classes u wont need to miss lessons cause you can get everything set up and go t

https://towardsdatascience.com/how-to-apply-transformers-to-any-length-of-text-a5601410af7f

In [47]:
def breakintochunks(input_id_chunks, mask_chunks):
    input_id_chunks_ = list(input_id_chunks)
    mask_chunks_ = list(mask_chunks)

    for i in range(len(input_id_chunks_)):
      input_id_chunks_[i] = torch.cat([
          torch.Tensor([101]), input_id_chunks_[i], torch.Tensor([102])
          ])
      mask_chunks_[i] = torch.cat([
          torch.Tensor([1]), mask_chunks_[i], torch.Tensor([1])
          ])
      
      pad_len = max_sequence_length - input_id_chunks_[i].shape[0]
      if pad_len > 0:
        input_id_chunks_[i] = torch.cat([
            input_id_chunks_[i], torch.Tensor([0] * pad_len)
        ])

        mask_chunks_[i] = torch.cat([
            mask_chunks_[i], torch.Tensor([0] * pad_len)
        ])
      print(len(input_id_chunks_[i]), end = ' ')
    print()

    input_id = torch.stack(input_id_chunks_)
    attention_mask = torch.stack(mask_chunks_)

    return input_id, attention_mask

max_len = 0
max_sequence_length = 512
input_ids = []
attention_masks = []

for k in range(len(sentences)):
    sent = sentences[k]
    tokens = tokenizer.encode_plus(sent, add_special_tokens=False, return_tensors='pt')
    input_id_chunks = tokens['input_ids'][0].split(max_sequence_length - 2)
    mask_chunks = tokens['attention_mask'][0].split(max_sequence_length - 2)

    input_id, attention_mask = breakintochunks(input_id_chunks, mask_chunks)

    input_ids.append(input_id)
    attention_masks.append(attention_mask)

    if k > 10:
      break

#print('Max paragraph length: ', max_len)
input_ids = torch.cat(input_ids, dim = 0)
attention_masks = torch.cat(attention_masks, dim =0)
labels = torch.tensor(labels)

print(input_ids[0])


512 
512 
512 
512 512 
512 
512 
512 
512 
512 512 
512 
512 
512 
tensor([  101.,  1045.,  2228.,  2008.,  2493.,  2052.,  5770.,  2013.,  4083.,
         2012.,  2188.,  1010.,  2138.,  2027.,  2180.,  2102.,  2031.,  2000.,
         2689.,  1998.,  2131.,  2039.,  2220.,  1999.,  1996.,  2851.,  2000.,
         6457.,  1998.,  2079.,  2045.,  2606.,  1012.,  2635.,  2069.,  4280.,
         7126.,  2068.,  2138.,  2012.,  2045.,  2160.,  2027.,  1005.,  2222.,
         2022.,  3477.,  2062.,  3086.,  1012.,  2027.,  2097.,  2022.,  6625.,
         2012.,  2188.,  1012.,  1996., 18263.,  2112.,  1997.,  2082.,  2003.,
         2893.,  3201.,  1012.,  2017.,  5256.,  2039.,  2175.,  8248.,  2115.,
         4091.,  1998.,  2175.,  2000.,  2115.,  9346.,  1998.,  2298.,  2012.,
         2115.,  8416.,  2015.,  1012.,  2044.,  2017.,  2228.,  2017.,  3856.,
         1037., 11018.,  1057.,  2175.,  2298.,  1999.,  1996.,  5259.,  1998.,
         2017.,  3363.,  2593.,  2025.,  2066.,  200

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:52: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
